In [23]:
import json
import requests
import os

# os.environ['NO_PROXY'] = '127.0.0.1'
url = "http://127.0.0.1:5000"

## Get info


In [24]:
response = requests.post(url + "/routeGetDatasetsInfo", {})
dataset = 'wesad_dimensional'
# dataset = 'wesad_dimensional'
json.loads(response.content)

{'loadedDatasetsIds': [],
 'localDatasetsIds': ['case_categorical',
  'case_dimensional',
  'wesad_dimensional',
  'wesad_categorical']}

## Load local dataset

In [25]:
response = requests.post(url + "/loadLocalDataset",data = {'datasetId':dataset})
json.loads(response.content)

{'state': 'success'}

## Remove dataset

In [17]:
response = requests.post(url + "/removeDataset",data = {'datasetId':dataset})
json.loads(response.content)

{'state': 'error'}

## Get Dataset info

In [20]:
response = requests.post(url + "/getDatasetInfo",data = {'datasetId':dataset})

info = json.loads(response.content)
info

{'categoricalLabels': {'Dominant hand': ['left', 'right'],
  'Gender': ['female', 'male']},
 'dimensions': {'Arousal': 'arousal', 'Valence': 'valence'},
 'globalEmotionMax': {'Arousal': 9.0, 'Valence': 9.0},
 'globalEmotionMin': {'Arousal': 0.0, 'Valence': 0.0},
 'ids': ['s10',
  's11',
  's13',
  's14',
  's15',
  's16',
  's17',
  's2',
  's3',
  's4',
  's5',
  's6',
  's7',
  's8',
  's9'],
 'instanceLen': 15,
 'isDated': False,
 'labels': ['Base', 'TSST', 'Medi 1', 'Fun', 'Medi2'],
 'numericalLabels': ['Age', 'Height (cm)', 'Weight (kg)'],
 'seriesLabels': ['Valence', 'Arousal'],
 'temporalLen': 5,
 'type': 'dimensional',
 'variablesLen': 32}

## Get Time Series

In [27]:
response = requests.post(url + "/getMTSeries",data = {'datasetId':dataset, 'begin': 0, 'end': 5, 'ids': json.dumps(['s4', 's2', 's5'])})
json.loads(response.content)

{'s2': {'categoricalFeatures': ['right', 'male'],
  'categoricalLabels': ['Dominant hand', 'Gender'],
  'index': ['0', '1', '2', '3', '4'],
  'metadata': {'id': 's2'},
  'numericalFeatures': [27.0, 175.0, 80.0],
  'numericalLabels': ['Age', 'Height (cm)', 'Weight (kg)'],
  'temporalVariables': {'Arousal': [2.0, 4.0, 2.0, 1.0, 2.0],
   'PANAS-Active': [1.0, 3.0, 1.0, 1.0, 1.0],
   'PANAS-Afraid': [2.0, 3.0, 1.0, 1.0, 1.0],
   'PANAS-Alert': [4.0, 5.0, 2.0, 4.0, 2.0],
   'PANAS-Annoyed': [1.0, 3.0, 1.0, 1.0, 1.0],
   'PANAS-Ashamed': [1.0, 1.0, 1.0, 1.0, 1.0],
   'PANAS-Attentive': [4.0, 4.0, 3.0, 3.0, 2.0],
   'PANAS-Determined': [4.0, 4.0, 2.0, 1.0, 1.0],
   'PANAS-Distressed': [1.0, 2.0, 1.0, 1.0, 1.0],
   'PANAS-Enthusiastic': [2.0, 3.0, 3.0, 2.0, 2.0],
   'PANAS-Excited': [2.0, 4.0, 1.0, 1.0, 1.0],
   'PANAS-Frustrated': [1.0, 3.0, 1.0, 2.0, 1.0],
   'PANAS-Guilty': [1.0, 1.0, 1.0, 1.0, 1.0],
   'PANAS-Happy': [2.0, 2.0, 4.0, 3.0, 3.0],
   'PANAS-Hostile': [1.0, 1.0, 1.0, 1.0, 1.0],

## Get projection


In [49]:
alphas = {label: 1 for label in info["seriesLabels"]}
D_k = {}
coords = {}

In [50]:

response = requests.post(url + "/getDatasetProjection",data = {'datasetId':dataset, 'alphas': json.dumps(alphas), 'begin': 0, 'end':40, 
                                                               'D_k': json.dumps(D_k), 'oldCoords': json.dumps(coords)})
data = json.loads(response.content)
coords = data["coords"]
D_k = data["D_k"]
coords

{'1': [10.331422178159485, -7.822703821703016],
 '2': [9.149584635408338, 1.78141223125199],
 '3': [-10.179948481847532, -1.4284377541435411],
 '4': [-3.7285033656149005, -10.076584413672705],
 '5': [-7.418150277018119, 9.925811489772995],
 '6': [1.845595310912727, 7.620502268494276]}

## Clustering

In [38]:
response = requests.post(url + "/doClustering",data = {'datasetId':dataset, 'coords': json.dumps(coords), 'k': 2})
clusters = json.loads(response.content)
clusters

{'0': ['1', '2', '4', '5'], '1': ['3', '6']}

## Rankings

In [39]:
response = requests.post(url + "/getFishersDiscriminantRanking",data = {'datasetId':dataset, 'D_k': json.dumps(D_k), 'blueCluster': json.dumps(clusters['0']), 'redCluster': json.dumps(clusters['1']) })
j_s = json.loads(response.content)
j_s

{'Arousal': 0.051812307045928346, 'Valence': 3.415525686489257}

## Temporal summary

In [32]:
response = requests.post(url + "/getTemporalSummary",data = {'datasetId':dataset})
# json.loads(response.content)["mean"]


## Downsample

In [36]:
response = requests.post(url + "/downsampleData",data = {'datasetId':dataset, 'rule': 'T'})
json.loads(response.content)

{'state': 'success'}

## Reset dataset

In [68]:
response = requests.post(url + "/resetDataset",data = {'datasetId':dataset})
json.loads(response.content)

{'state': 'success'}